<a href="https://colab.research.google.com/github/alhichri/RS-DeepSuperLearner/blob/main/CNN_fusion_methods_tfrecords_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a test file to learn how to use colab

In [ ]:
!pip install keras_efficientnets
!pip install keras_applications


import numpy as np
import scipy.io
import matplotlib.pyplot as plt
import itertools
import seaborn as sn
import time
import pathlib
import glob
import itertools

from sklearn.metrics import confusion_matrix,  classification_report, accuracy_score
from sklearn.utils.multiclass import unique_labels
from sklearn.utils import shuffle

import tensorflow as tf
tf.random.set_seed(1)

# from tensorflow import set_random_seed
# set_random_seed(1)
from tensorflow.python.keras import backend as k   # added
from tensorflow.keras import utils as np_utils   #added

from tensorflow.keras import layers
from tensorflow import keras 
from tensorflow.keras.applications import inception_v3,inception_resnet_v2, densenet, vgg16
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.imagenet_utils import preprocess_input, decode_predictions

from tensorflow.keras.models import model_from_json

from tensorflow.python.client import device_lib
device_lib.list_local_devices()
!python -V

Python 3.7.12


In [ ]:
import keras_efficientnets 
from keras_efficientnets import efficientnet
from keras_efficientnets.custom_objects import Swish
from google.colab import drive
drive.mount('/content/gdrive' , force_remount=True)


Mounted at /content/gdrive


In [ ]:
# from importlib.machinery import SourceFileLoader
# somemodule = SourceFileLoader('squeezenet', '/content/gdrive/My Drive/utils/squeezenet.py').load_module()
# #somemodule = SourceFileLoader('squeezenet', '/content/gdrive/My Drive/students/ruwaida_alharbi/colab_folder/utils/squeezenet.py').load_module()
# from squeezenet import SqueezeNet
# #DNet1 = SqueezeNet(input_shape=img_dim, include_top=True,weights='imagenet')

def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    ## As loss always exists
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    
    ## Loss
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    ## Accuracy
    plt.figure(2)
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')

    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

 
    
def plot_confusion_matrix2(cm, title='Confusion matrix', cmap=plt.cm.Oranges):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(cm.shape[1])
    plt.xticks(tick_marks, rotation=45)
    ax = plt.gca()
    ax.set_xticklabels((ax.get_xticks() +1).astype(str))
    plt.yticks(tick_marks)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

def load_training_model(dataset, trainSize , fold_number, model_name):
    DS = 'dataset%i_' %dataset
    fname = '_trainSize%i_' %trainSize
    folder_name = 'fold%i' %fold_number
    fname_with_path = '/content/gdrive/My Drive/RS_results/' + DS + 'best_'+ model_name  + fname + folder_name
    # load json and create model
    json_file = open(fname_with_path+".json", 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    theModel= model_from_json(loaded_model_json)
    # load weights into new model
    theModel.load_weights(fname_with_path+".h5")
    print("Loaded theModel from disk: ", fname_with_path)

    return theModel

def save_training_results(dataset, trainSize, model_name,  y_validation_prob0,y_validation_prob1, y_test_prob):
    DS = 'dataset%i_' %dataset
    fname = 'trainSize%i' %trainSize
    fname_with_path = '/content/gdrive/My Drive/RS_results/' + DS + 'res_' + model_name + '_'+ fname+'_validation_and_test'
    D = {'y_validation_prob0': y_validation_prob0, 'y_validation_prob1': y_validation_prob1, 'y_test_prob': y_test_prob}
    np.savez(fname_with_path, **D)  
    print("Saved results to disk: ", fname_with_path)

def load_training_results(dataset, model_name, trainSize, fold_number  ):
    DS = 'dataset%i_' %dataset
    fname = 'trainSize%i' %trainSize
    fname_with_path = '/content/gdrive/My Drive/RS_results/' + DS + 'res_' + model_name + '_'+ fname+  '_fold_'+ str(fold_number) + '.npz'
    D = np.load(fname_with_path)
    y_test = D['y_test']; y_prob = D['y_prob'];  

    print("loaded results from disk: ", fname_with_path)
    return y_prob[1], y_test


In [ ]:
# import pathlib
# data_dir = tf.keras.utils.get_file(origin='https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
#                                          fname='flower_photos', untar=True)
# print(data_dir)

# data_dir = pathlib.Path(data_dir)
# print(data_dir)


dataset=5; data_dir = '/content/gdrive/My Drive/RS_datasets/AID'
tfrecords_dir = '/content/gdrive/My Drive/RS_datasets/AID_tfrecords/'
subset = 'train'
batch_size = 32
CLASS_NAMES = [  'airport' , 'bare land' , 'baseball field' , 'beach' , 'bridge' , 'center' , 'church' , 'commercial' , 'dense residential' ,  \
'desert' , 'farmland' , 'forest' , 'industrial' , 'meadow' , 'medium residential' , 'mountain' , 'park' , 'parking' , 'playground' ,    \
'pond' , 'port' , 'railway station' , 'resort' , 'river' , 'school' , 'sparse residential' , 'square' , 'stadium' , 'storage tanks' , 'viaduct'  ]
print('CLASS_NAMES: ' , CLASS_NAMES  )

# dataset=6;  data_dir = '/content/gdrive/My Drive/RS_datasets/NWPU_RESISC45'
# tfrecords_dir = '/content/gdrive/My Drive/RS_datasets/NWPU_tfrecords/'
# subset = 'train'
# batch_size = 32

# dataset=7;  data_dir = '/content/gdrive/My Drive/RS_datasets/PatternNet'
# tfrecords_dir = '/content/gdrive/My Drive/RS_datasets/PatternNet_tfrecords/'
# subset = 'train'
# batch_size = 32

data_dir = pathlib.Path(data_dir)
print(data_dir)


image_count = len(list(data_dir.glob('*/*.png')))
print('Total number of images in dataset: ' , image_count)
CLASS_NAMES0 = np.array([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"])

num_classes = len(CLASS_NAMES0)
print(CLASS_NAMES0)
print(num_classes)


CLASS_NAMES:  ['airport', 'bare land', 'baseball field', 'beach', 'bridge', 'center', 'church', 'commercial', 'dense residential', 'desert', 'farmland', 'forest', 'industrial', 'meadow', 'medium residential', 'mountain', 'park', 'parking', 'playground', 'pond', 'port', 'railway station', 'resort', 'river', 'school', 'sparse residential', 'square', 'stadium', 'storage tanks', 'viaduct']
/content/gdrive/My Drive/RS_datasets/AID
Total number of images in dataset:  0
[]
0


In [ ]:


k = 0
for c in range(len(CLASS_NAMES)):
  if CLASS_NAMES0[k]!= CLASS_NAMES[k]:
    print('Classe names different: ' , CLASS_NAMES0[k] , ' and ', CLASS_NAMES[k])
    k = k +1

# trainSize = 10
# fold_number = 99
# print(dataset)
# print(trainSize)

# y_prob , y_test = load_training_results(dataset=dataset, model_name='VGG16', trainSize=trainSize, fold_number=fold_number    )
# # dataset5_res_Inception_resnet_v2_trainSize20_fold_99


# Test_predicted = np.argmax(y_prob[1], axis=1)
# OA1 = accuracy_score(y_test,Test_predicted)
# print('---->  test set with trainsize :', trainSize , ' % fold : ', fold_number,  ' Accuracy: ' , OA1)



In [ ]:
#################################################################################################
#######  Load prediction results
#######  for a given training size
trainSize = 20
#################################################################################################


def single_class_accuracy(class_id_true, class_id_preds , INTERESTING_CLASS_ID):
    accuracy_mask = tf.keras.backend.cast(tf.keras.backend.equal(class_id_preds, INTERESTING_CLASS_ID), 'int32')
    class_acc_tensor = tf.keras.backend.cast(tf.keras.backend.equal(class_id_true, class_id_preds), 'int32') * accuracy_mask
    class_acc = tf.keras.backend.sum(class_acc_tensor) / tf.keras.backend.maximum(tf.keras.backend.sum(accuracy_mask), 1)
    return class_acc


y0 , y_fold0 = load_training_results(dataset=dataset, model_name='VGG16', trainSize=trainSize, fold_number=0    )
y1 , y_fold1 = load_training_results(dataset=dataset, model_name='VGG16', trainSize=trainSize, fold_number=1    )
y_test_prob1 , y_test1 = load_training_results(dataset=dataset, model_name='VGG16', trainSize=trainSize, fold_number=99    )
y_validation_prob1 = np.vstack((y0, y1 ))
print(y0.shape  , y1.shape, y_validation_prob1.shape)

y0 , y_fold0 = load_training_results(dataset=dataset, model_name='InceptionV3', trainSize=trainSize, fold_number=0    )
y1 , y_fold1 = load_training_results(dataset=dataset, model_name='InceptionV3', trainSize=trainSize, fold_number=1    )
y_test_prob2 , y_test2 = load_training_results(dataset=dataset, model_name='InceptionV3', trainSize=trainSize, fold_number=99    )
y_validation_prob2 = np.vstack((y0, y1 ))
print(y0.shape  , y1.shape, y_validation_prob2.shape)


y0 , y_fold0 = load_training_results(dataset=dataset, model_name='Inception_resnet_v2', trainSize=trainSize, fold_number=0    )
y1 , y_fold1 = load_training_results(dataset=dataset, model_name='Inception_resnet_v2', trainSize=trainSize, fold_number=1    )
y_test_prob3 , y_test3 = load_training_results(dataset=dataset, model_name='Inception_resnet_v2', trainSize=trainSize, fold_number=99    )
y_validation_prob3 = np.vstack((y0, y1 ))
print(y0.shape  , y1.shape, y_validation_prob3.shape)

y0 , y_fold0 = load_training_results(dataset=dataset, model_name='DenseNet121', trainSize=trainSize, fold_number=0    )
y1 , y_fold1 = load_training_results(dataset=dataset, model_name='DenseNet121', trainSize=trainSize, fold_number=1    )
y_test_prob4 , y_test4 = load_training_results(dataset=dataset, model_name='DenseNet121', trainSize=trainSize, fold_number=99    )
y_validation_prob4 = np.vstack((y0, y1 ))
print(y0.shape  , y1.shape, y_validation_prob4.shape)

y0 , y_fold0 = load_training_results(dataset=dataset, model_name='efficientnetB3', trainSize=trainSize, fold_number=0    )
y1 , y_fold1 = load_training_results(dataset=dataset, model_name='efficientnetB3', trainSize=trainSize, fold_number=1    )
y_test_prob5 , y_test5 = load_training_results(dataset=dataset, model_name='efficientnetB3', trainSize=trainSize, fold_number=99    )
y_validation_prob5 = np.vstack((y0, y1 ))
print(y0.shape  , y1.shape, y_validation_prob5.shape)

y_fold0_and_1 = np.hstack((y_fold0, y_fold1 ))
print(y_fold0.shape  , y_fold1.shape, y_fold0_and_1.shape)

Test_predicted1 = np.argmax(y_validation_prob1, axis=1); OA1_val = accuracy_score(y_fold0_and_1,Test_predicted1)
acc1 = np.zeros(( num_of_classes  ))
for cn in range(num_of_classes):
    acc1[cn] = single_class_accuracy(y_fold0_and_1, Test_predicted1 , cn)
print(acc1)
Test_predicted2 = np.argmax(y_validation_prob2, axis=1); OA2_val = accuracy_score(y_fold0_and_1,Test_predicted2)
acc2 = np.zeros(( num_of_classes  ))
for cn in range(num_of_classes):
    acc2[cn] = single_class_accuracy(y_fold0_and_1, Test_predicted2 , cn)
print(acc2)
Test_predicted3 = np.argmax(y_validation_prob3, axis=1); OA3_val = accuracy_score(y_fold0_and_1,Test_predicted3)
acc3 = np.zeros(( num_of_classes  ))
for cn in range(num_of_classes):
    acc3[cn] = single_class_accuracy(y_fold0_and_1, Test_predicted3 , cn)
print(acc3)
Test_predicted4 = np.argmax(y_validation_prob4, axis=1); OA4_val = accuracy_score(y_fold0_and_1,Test_predicted4)
acc4 = np.zeros(( num_of_classes  ))
for cn in range(num_of_classes):
    acc4[cn] = single_class_accuracy(y_fold0_and_1, Test_predicted4 , cn)
print(acc4)
Test_predicted5 = np.argmax(y_validation_prob5, axis=1); OA5_val = accuracy_score(y_fold0_and_1,Test_predicted5)
acc5 = np.zeros(( num_of_classes  ))
for cn in range(num_of_classes):
    acc5[cn] = single_class_accuracy(y_fold0_and_1, Test_predicted5 , cn)
print(acc5)

print('acc of all models on validation folds : ' , OA1_val ,  OA2_val , OA3_val ,  OA4_val ,OA5_val ) 

# for i in range(y_test1.shape[0]):
#   if y_test5[i]!=y_test2[i]:
#     print('whoops they are different!')



FileNotFoundError: ignored

In [ ]:
y_validation_prob1.shape

(4980, 30)

In [ ]:
y_test = y_test1

Test_predicted1 = np.argmax(y_test_prob1, axis=1); OA1 = accuracy_score(y_test,Test_predicted1)
Test_predicted2 = np.argmax(y_test_prob2, axis=1); OA2 = accuracy_score(y_test,Test_predicted2)
Test_predicted3 = np.argmax(y_test_prob3, axis=1); OA3 = accuracy_score(y_test,Test_predicted3)
Test_predicted4 = np.argmax(y_test_prob4, axis=1); OA4 = accuracy_score(y_test,Test_predicted4)
Test_predicted5 = np.argmax(y_test_prob5, axis=1); OA5 = accuracy_score(y_test,Test_predicted5)
print('acc of all models on test set  : ' , OA1 ,  OA2 , OA3 ,  OA4 ,OA5 ) 

Num_of_classifiers = 5

pred_status = np.zeros(( len(Test_predicted1) , Num_of_classifiers  ))  
for i in range(len(y_test)):
    pred_status[i,0] = Test_predicted1[i] == y_test[i]
    pred_status[i,1] = Test_predicted2[i] == y_test[i]
    pred_status[i,2] = Test_predicted3[i] == y_test[i]
    pred_status[i,3] = Test_predicted4[i] == y_test[i]
    pred_status[i,4] = Test_predicted5[i] == y_test[i]

number_of_all5correct = np.sum( np.sum(pred_status , axis=1)==5  )
number_of_atleast1correct = np.sum( np.sum(pred_status , axis=1)>=1  )
number_of_0correct = np.sum( np.sum(pred_status , axis=1)==0  )
maximum_accuracy = (len(Test_predicted1) - number_of_0correct) / len(Test_predicted1)
print('len(Test_predicted1)=' , len(Test_predicted1))
print('number_of_all5correct=' , number_of_all5correct)
print('number_of_atleast1correct=' , number_of_atleast1correct)
print('number_of_0correct=' , number_of_0correct)
print('maximum_accuracy=' , maximum_accuracy)

print(OA1); print(OA2); print(OA3); print(OA4); print(OA5);  print(maximum_accuracy)





acc of all models on test set  :  0.8554216867469879 0.8715975011155734 0.8852074966532798 0.8982597054886211 0.9154395359214637
len(Test_predicted1)= 8964
number_of_all5correct= 6609
number_of_atleast1correct= 8731
number_of_0correct= 233
maximum_accuracy= 0.9740071396697902
0.8554216867469879
0.8715975011155734
0.8852074966532798
0.8982597054886211
0.9154395359214637
0.9740071396697902


In [ ]:
def print_array(dataset_OA_avg , dataset_OA_std, column_labels):

    for i in range(len(column_labels)-1):
        print('{:4.2f}+{:.7}'.format(0 , column_labels[i] ) , end ="\t" )
    print('{:4.2f}+{:.7}'.format(0 , column_labels[i+1] ) )
    for i in range(dataset_OA_avg.shape[0]):
        for j in range(dataset_OA_avg.shape[1]-1):
            #print(' {:.3f} '.format(probabilities[i])   ),
            print('{:5.2f}-{:.2f}'.format(dataset_OA_avg[i,j] , dataset_OA_std[i,j] ) , end ="\t" )
        print('{:5.2f}-{:.2f}'.format(dataset_OA_avg[i,j+1] , dataset_OA_std[i,j+1] ) )

################################################
#  fusion based  naive averaging 

# y_test_prob1_new = y_test_prob1.T / np.max(y_test_prob1, axis=1)
# y_test_prob2_new = y_test_prob2.T  / np.max(y_test_prob2, axis=1)
# y_test_prob3_new = y_test_prob3.T  / np.max(y_test_prob3, axis=1)
# y_test_prob4_new = y_test_prob4.T  / np.max(y_test_prob4, axis=1)
# y_test_prob5_new = y_test_prob5.T  / np.max(y_test_prob5, axis=1)

# y_test_prob1_new = y_test_prob1_new.T 
# y_test_prob2_new = y_test_prob2_new.T  
# y_test_prob3_new = y_test_prob3_new.T  
# y_test_prob4_new = y_test_prob4_new.T  
# y_test_prob5_new = y_test_prob5_new.T  

y_prob = np.zeros(  y_test_prob1.shape  )
for i in range(len(y_test)):
    y_prob[i,:] = (  y_test_prob1[i,:] + y_test_prob2[i,:] + y_test_prob3[i,:] + y_test_prob4[i,:] + y_test_prob5[i,:]   ) / Num_of_classifiers

Test_predicted = np.argmax(y_prob, axis=1)
cm_after_adaptation = confusion_matrix(y_test, Test_predicted)
OA_fused_naive = accuracy_score(y_test,Test_predicted)
print('---->  Average prob fusion with testing size %i percent' % trainSize)
print('acc 1 to 5, fused acc : ' , OA1 ,  OA2 , OA3 ,  OA4 ,OA5 ,OA_fused_naive) 


# print('y_test_prob.shape: ' , y_test_prob1.shape)
# column_labels = [' AVG', ' SEN' , ' SPE' , ' PRE' ,  ' F1' , ' Gmean', 'HL',  'AP', 'HL', 'RL']
# y_test_prob1_std = np.zeros(y_test_prob1.shape)
# print_array(y_test_prob1[:10] , y_test_prob1_std[:10], column_labels)
print(OA1); print(OA2); print(OA3); print(OA4); print(OA5);  print(OA_fused_naive)


---->  Average prob fusion with testing size 10 percent
acc 1 to 5, fused acc :  0.8554216867469879 0.8715975011155734 0.8852074966532798 0.8982597054886211 0.9154395359214637 0.930611334225792
0.8554216867469879
0.8715975011155734
0.8852074966532798
0.8982597054886211
0.9154395359214637
0.930611334225792


In [ ]:
################################################
#  fusion based  weighted averaging 
probabilities = np.vstack(( acc1  , acc2))
probabilities = np.vstack(( probabilities  , acc3))
probabilities = np.vstack(( probabilities  , acc4))
probabilities = np.vstack(( probabilities  , acc5))
print(probabilities.shape)  # num of classifiers versus number of classes
print(np.max(probabilities, axis=0))   
# probabilities = probabilities / np.max(probabilities, axis=0)
print(np.exp(probabilities))

probabilities = np.exp(probabilities) / np.sum(np.exp(probabilities), axis=0)
probabilities = probabilities.T
y_prob = np.zeros(  y_test_prob1.shape  )
for i in range(len(y_test)):
  for c in range(num_of_classes):
    y_prob[i,c]  =   probabilities[c,0]*y_test_prob1[i,c] + \
    probabilities[c,1]*y_test_prob2[i,c] + \
    probabilities[c,2]*y_test_prob3[i,c] + \
    probabilities[c,3]*y_test_prob4[i,c] + \
    probabilities[c,4]*y_test_prob5[i,c]   
print(y_prob)

Test_predicted = np.argmax(y_prob, axis=1)
cm_after_adaptation = confusion_matrix(y_test, Test_predicted)
OA_fused_weighted = accuracy_score(y_test,Test_predicted)
print('---->  Weighted average prob fusion with testing size %i percent' % trainSize)
print('acc 1 to 5, fused acc : ' , OA1 ,  OA2 , OA3 ,  OA4 ,OA5 ,OA_fused_weighted) 

print(OA1); print(OA2); print(OA3); print(OA4); print(OA5);  print(OA_fused_weighted)




(5, 30)
[0.84615385 0.96153846 0.93333333 0.93939394 0.96296296 0.88888889
 1.         0.74193548 1.         0.92857143 0.78571429 0.94594595
 0.91111111 0.94444444 0.95238095 0.77419355 0.89473684 0.96296296
 1.         0.825      1.         0.96969697 0.96666667 1.
 0.77142857 0.83333333 0.86363636 0.94285714 1.         0.975     ]
[[2.31136403 2.1330988  2.44395352 2.41089971 2.19872751 1.97746987
  2.40797917 1.5754571  2.35641844 1.742909   1.81076607 2.35641844
  2.34822221 2.43242545 2.53509867 1.74741846 2.43974101 2.43242545
  2.27852752 1.87842711 2.49189503 2.49499248 2.44209711 2.33610462
  2.01375271 1.99302816 2.08932721 2.38314502 2.37176965 2.45330452]
 [2.04272707 2.04272707 2.39887529 2.40797917 2.61944631 1.8769153
  2.62916595 1.6738921  2.20493422 2.53089104 2.06295193 2.4887105
  2.02122488 2.55358946 2.59187345 1.75983031 2.35641844 2.35641844
  2.62614566 2.05022331 2.22554093 2.2716645  2.21024517 2.16566456
  1.92292248 2.30097589 2.25353479 2.46755013 2.59748

In [ ]:
################################################
#  fusion based  on choosing the classifier with the highest validation prob
Test_predicted_all = np.zeros(( len(y_test) , Num_of_classifiers))
Test_predicted_all[:,0] = Test_predicted1
Test_predicted_all[:,1] = Test_predicted2
Test_predicted_all[:,2] = Test_predicted3
Test_predicted_all[:,3] = Test_predicted4
Test_predicted_all[:,4] = Test_predicted5


probabilities = np.vstack(( acc1  , acc2))
probabilities = np.vstack(( probabilities  , acc3))
probabilities = np.vstack(( probabilities  , acc4))
probabilities = np.vstack(( probabilities  , acc5))
# print(probabilities.T)
max_prob = np.max(probabilities , axis=0)


print('y_test_prob1.shape: ' , y_test_prob1.shape)
Test_predicted = y_test.copy()
confidence_in_predicted_class = np.zeros((  len(y_test) , Num_of_classifiers ))
for i in range(len(y_test)):
  predicted_class = Test_predicted1[i];   confidence_in_predicted_class[i , 0] = probabilities[0,predicted_class]
  predicted_class = Test_predicted2[i];   confidence_in_predicted_class[i , 1] = probabilities[1,predicted_class]
  predicted_class = Test_predicted3[i];   confidence_in_predicted_class[i , 2] = probabilities[2,predicted_class]
  predicted_class = Test_predicted4[i];   confidence_in_predicted_class[i , 3] = probabilities[3,predicted_class]
  predicted_class = Test_predicted5[i];   confidence_in_predicted_class[i , 4] = probabilities[4,predicted_class]
  classifier_with_highest_confidence = np.argmax(confidence_in_predicted_class[i , :])
  Test_predicted[i] =  Test_predicted_all[i ,  classifier_with_highest_confidence ]
  # if Test_predicted[i] !=y_test[i]:
    # print('image '+str(i)+  ' true label: ' + str(y_test[i])+ CLASS_NAMES[y_test[i]]  )
      # print('image '+str(i)+  ' true label: ' + str(y_test[i])+ CLASS_NAMES[y_test[i]] + ' but predicted: ' + str( Test_predicted[i] )+ CLASS_NAMES[Test_predicted[i]] +  str( Test_predicted_all[i,:]    ) + str(  confidence_in_predicted_class[i,:]   ) )
      # print('Whoops! image '+str(i)+  ' true label: ' + str(y_test[i]) +str(Test_predicted_all[i,:]   )+ ' but predicted: ' + str( Test_predicted[i] ) + str( Test_predicted_all[i,:]    ) ) 

OA_fused = accuracy_score(y_test,Test_predicted)
print('---->  Weighted average prob fusion with testing size %i percent' % trainSize)
print('acc 1 to 5, fused acc : ' , OA1 ,  OA2 , OA3 ,  OA4 ,OA5 ,OA_fused) 



y_test_prob1.shape:  (8964, 30)
---->  Weighted average prob fusion with testing size 10 percent
acc 1 to 5, fused acc :  0.8554216867469879 0.8715975011155734 0.8852074966532798 0.8982597054886211 0.9154395359214637 0.8912315930388219


In [ ]:
i=428
plt.imshow(X_test[i])
# plt.title(    'true label: ' + CLASS_NAMES[y_test[i]] +str(probabilities[:,y_test[i]])+ ' but majority said: ' + CLASS_NAMES[Test_predicted[i]] + str( probabilities[:,Test_predicted[i]])        )
plt.title(    'image '+str(i)+  ' true label: ' + str(y_test[i])+ CLASS_NAMES[y_test[i]] + ' but majority said: ' + str( Test_predicted[i] )+ CLASS_NAMES[Test_predicted[i]] +  str( Test_predicted_all[i,:]    ) + str(  confidence_in_predicted_class[i,:]       )         )
plt.axis('off')



In [ ]:
################################################
#  fusion based on majority voting
Test_predicted_all = np.zeros(( len(y_test) , Num_of_classifiers))
Test_predicted_all[:,0] = Test_predicted1
Test_predicted_all[:,1] = Test_predicted2
Test_predicted_all[:,2] = Test_predicted3
Test_predicted_all[:,3] = Test_predicted4
Test_predicted_all[:,4] = Test_predicted5
# Test_predicted_all[:,3] = Test_predicted4
# Test_predicted_all[:,4] = Test_predicted5
#print(Test_predicted_all)
#print(y_test)
print(num_of_classes)

freq_of_curr_class = np.zeros(( len(y_test)  , num_of_classes ))
for c in range(num_of_classes):
    temp_equalities = Test_predicted_all==c  # find all predictions that are equal to current class, predictions start from 1 not 0
    freq_of_curr_class[:,c] = np.sum(temp_equalities , axis=1)  # compute frequencies of each class
#print(freq_of_curr_class.shape)
print(freq_of_curr_class[0 , :])
Test_predicted_max = np.max(freq_of_curr_class, axis=1) # Find the maximum frequency
#print(Test_predicted_max) 
Test_predicted = np.argmax(freq_of_curr_class, axis=1)      #  Find the class with the maximum frequency
#print(Test_predicted)
cm_after_adaptation = confusion_matrix(y_test, Test_predicted)
OA_fused_majority = accuracy_score(y_test,Test_predicted)
print('---->  majority voting  fusion with testing size %i percent' % trainSize)
print('acc 1 to 5, fused acc : ' , OA1 ,  OA2 , OA3 ,  OA4 ,OA5 ,OA_fused_majority) 


30
[5. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
---->  majority voting  fusion with testing size 10 percent
acc 1 to 5, fused acc :  0.8554216867469879 0.8715975011155734 0.8852074966532798 0.8982597054886211 0.9154395359214637 0.9253681392235609
acc 1 to 5, fused acc :  0.8554216867469879 0.8715975011155734 0.8852074966532798 0.8982597054886211 0.9154395359214637 0.9253681392235609


In [ ]:
################################################
#  fusion based on super learner from probablities to predicted classes
from sklearn.linear_model import LogisticRegression

print('acc1.shape: ' ,  acc1.shape)
print('acc of all models on validation folds : ' , OA1_val ,  OA2_val , OA3_val ,  OA4_val ,OA5_val ) 

val_acc1 = np.hstack((acc1 , OA1_val))
val_acc2 = np.hstack((acc2 , OA2_val))
val_acc3 = np.hstack((acc3 , OA3_val))
val_acc4 = np.hstack((acc4 , OA4_val))
val_acc5 = np.hstack((acc5 , OA5_val))

probabilities1 = np.tile(val_acc1, [y_validation_prob1.shape[0] , 1])
probabilities2 = np.tile(val_acc2, [y_validation_prob2.shape[0] , 1])
probabilities3 = np.tile(val_acc3, [y_validation_prob3.shape[0] , 1])
probabilities4 = np.tile(val_acc4, [y_validation_prob4.shape[0] , 1])
probabilities5 = np.tile(val_acc5, [y_validation_prob5.shape[0] , 1])

print('probabilities1.shape: ' ,  probabilities1.shape)
print('probabilities1: ' ,  probabilities1)
print('probabilities2: ' ,  probabilities2)
print('probabilities3: ' ,  probabilities3)

fold_yhats =[]
fold_yhats.append(y_validation_prob1); #fold_yhats.append(probabilities1)  #; fold_yhats.append(probabilities1 * y_validation_prob1)
fold_yhats.append(y_validation_prob2); #fold_yhats.append(probabilities2)  #; fold_yhats.append(probabilities2 * y_validation_prob2)
fold_yhats.append(y_validation_prob3); #fold_yhats.append(probabilities3)  #; fold_yhats.append(probabilities3 * y_validation_prob3)
fold_yhats.append(y_validation_prob4); #fold_yhats.append(probabilities4)  #; fold_yhats.append(probabilities4 * y_validation_prob4)
fold_yhats.append(y_validation_prob5); #fold_yhats.append(probabilities5)  #; fold_yhats.append(probabilities5 * y_validation_prob5)
fold_yhats.append(probabilities1)  #; fold_yhats.append(probabilities1 * y_validation_prob1)
fold_yhats.append(probabilities2)  #; fold_yhats.append(probabilities2 * y_validation_prob2)
fold_yhats.append(probabilities3)  #; fold_yhats.append(probabilities3 * y_validation_prob3)
fold_yhats.append(probabilities4)  #; fold_yhats.append(probabilities4 * y_validation_prob4)
fold_yhats.append(probabilities5)  #; fold_yhats.append(probabilities5 * y_validation_prob5)

pred_prob_outputs = np.hstack(fold_yhats)
print('pred_prob_outputs.shape: ' ,  pred_prob_outputs.shape)
# Y_val = tf.keras.utils.to_categorical(y_val , num_classes )
# Y_val = np_utils.to_categorical(y_val , num_classes )
# fit a meta model
def fit_meta_model(X, y):
	model = LogisticRegression(random_state=2020, class_weight ='balanced',  verbose=1, multi_class='auto', tol= 1e-6)
	model.fit(X, y)
	return model



probabilities1 = np.tile(val_acc1, [y_test_prob1.shape[0] , 1])
probabilities2 = np.tile(val_acc2, [y_test_prob2.shape[0] , 1])
probabilities3 = np.tile(val_acc3, [y_test_prob3.shape[0] , 1])
probabilities4 = np.tile(val_acc4, [y_test_prob4.shape[0] , 1])
probabilities5 = np.tile(val_acc5, [y_test_prob5.shape[0] , 1])
fold_yhats =[]
fold_yhats.append(y_test_prob1); #fold_yhats.append(probabilities1)  #; fold_yhats.append(probabilities1 * y_test_prob1)
fold_yhats.append(y_test_prob2); #fold_yhats.append(probabilities2)  #; fold_yhats.append(probabilities2 * y_test_prob2)
fold_yhats.append(y_test_prob3); #fold_yhats.append(probabilities3)  #; fold_yhats.append(probabilities3 * y_test_prob3)
fold_yhats.append(y_test_prob4); #fold_yhats.append(probabilities4)  #; fold_yhats.append(probabilities4 * y_test_prob4)
fold_yhats.append(y_test_prob5); #fold_yhats.append(probabilities5)  #; fold_yhats.append(probabilities5 * y_test_prob5)
fold_yhats.append(probabilities1)  #; fold_yhats.append(probabilities1 * y_test_prob1)
fold_yhats.append(probabilities2)  #; fold_yhats.append(probabilities2 * y_test_prob2)
fold_yhats.append(probabilities3)  #; fold_yhats.append(probabilities3 * y_test_prob3)
fold_yhats.append(probabilities4)  #; fold_yhats.append(probabilities4 * y_test_prob4)
fold_yhats.append(probabilities5)  #; fold_yhats.append(probabilities5 * y_test_prob5)
test_prob_outputs = np.hstack(fold_yhats)




def fit_my_meta_model_v1(pred_prob_outputs, Y_val): # this model gets 88.5 for OPTIMAL31 at 20% training
	num_classes = Y_val.shape[1]
	inputs = tf.keras.layers.Input(shape=( pred_prob_outputs.shape[1]) )
	x = tf.keras.layers.Dense( pred_prob_outputs.shape[1] )(inputs)
	x = tf.keras.layers.Dropout(0.8)(x)
	x = tf.keras.layers.Dense( pred_prob_outputs.shape[1]  )(x)
	x = tf.keras.layers.Dropout(0.8)(x)
	outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
	metalernet_model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
	metalernet_model.summary()
	metalernet_model.compile(loss=['categorical_crossentropy'], \
			optimizer = tf.keras.optimizers.Adam(lr=0.001),metrics=['accuracy'])
	metalernet_model.fit(pred_prob_outputs, Y_val, epochs=90)
	metalernet_model.compile(loss=['categorical_crossentropy'], \
			optimizer = tf.keras.optimizers.Adam(lr=0.0001),metrics=['accuracy'])
	metalernet_model.fit(pred_prob_outputs, Y_val, epochs=15)
	metalernet_model.compile(loss=['categorical_crossentropy'], \
			optimizer = tf.keras.optimizers.Adam(lr=0.00001),metrics=['accuracy'])
	metalernet_model.fit(pred_prob_outputs, Y_val, epochs=10)
	return metalernet_model

def fit_my_meta_model(pred_prob_outputs, Y_val):
  num_classes = Y_val.shape[1]
  inputs = tf.keras.layers.Input(shape=( pred_prob_outputs.shape[1]) )
  x = tf.keras.layers.Dense( pred_prob_outputs.shape[1] )(inputs)
  # x = tf.keras.layers.BatchNormalization()(x)
  # x = tf.keras.layers.LeakyReLU(0.5)(x)
  # x = Swish()(x)
  x = tf.keras.layers.Dropout(0.8)(x)
  # x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Dense( pred_prob_outputs.shape[1] ,   activation='relu'  )(x)
  # x = tf.keras.layers.LeakyReLU(0.5)(x)
  x = tf.keras.layers.Dropout(0.8)(x)
  # x = tf.keras.layers.BatchNormalization()(x)
  # x = tf.keras.layers.Dense( pred_prob_outputs.shape[1]  )(x)
  # x = tf.keras.layers.LeakyReLU(0.5)(x)
  # x = tf.keras.layers.BatchNormalization()(x)
  # x = tf.keras.layers.Concatenate(  )([x0,x1])
  # x = tf.keras.layers.BatchNormalization()(x)
  # x = Swish()(x)
  # x0 = tf.keras.layers.Conv1D( filters=128 , kernel_size=1 , strides=1,   activation='relu')(inputs)
  # x = tf.keras.layers.BatchNormalization()(x0)
  # x = tf.keras.layers.GlobalAveragePooling1D()(x)
  outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
  metalernet_model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
  metalernet_model.summary()

  metalernet_model.compile(loss=['categorical_crossentropy'], \
      optimizer = tf.keras.optimizers.Adam(lr=0.001),metrics=['accuracy'])
  metalernet_model.fit(pred_prob_outputs, Y_val, epochs=300, batch_size=256, shuffle=True)
  metalernet_model.compile(loss=['categorical_crossentropy'], \
      optimizer = tf.keras.optimizers.Adam(lr=0.0001),metrics=['accuracy'])
  metalernet_model.fit(pred_prob_outputs, Y_val, epochs=15, batch_size=256, shuffle=True)
  # metalernet_model.compile(loss=['categorical_crossentropy'], \
  #     optimizer = tf.keras.optimizers.Adam(lr=0.00001),metrics=['accuracy'])
  # metalernet_model.fit(pred_prob_outputs, Y_val, epochs=10)
  return metalernet_model

y_fold0_and_1_categorical = np_utils.to_categorical(y_fold0_and_1 , num_classes )

y_test_prob_SL = []; OA = np.zeros(( 10  ))
for i in range(10):
    metalernet_model = fit_my_meta_model( pred_prob_outputs   ,  y_fold0_and_1_categorical )
    temp_result = metalernet_model.predict(pred_prob_outputs)
    y_validation_predicted =  np.argmax(temp_result  , axis=1   )  
    # y_validation_predicted = temp_result
    y_validation_predicted = y_validation_predicted.astype(np.int)
    OA_fused = accuracy_score(y_fold0_and_1,y_validation_predicted)
 

    y_test_prob_SL.append(   metalernet_model.predict(test_prob_outputs)    )
    y_test_predicted =  np.argmax(y_test_prob_SL[-1]  , axis=1   )       
    # y_test_predicted = temp_result
    y_test_predicted = y_test_predicted.astype(np.int)
    # print(y_test,y_test_predicted)
    OA[i] = accuracy_score(y_test,y_test_predicted)
    print('---->  superLearner fusion with validation set: ', OA_fused)
    print('acc 1 to 5, fused acc : ' , OA1 ,  OA2 , OA3 ,  OA4 ,OA5 , OA[i])    
    print(OA[i])
    
y_test_prob_SL_mean = np.zeros( y_test_prob_SL[0].shape  )
for i in range(len(y_test_prob_SL)):
  y_test_prob_SL_mean = y_test_prob_SL_mean + y_test_prob_SL[i]
y_test_prob_SL_mean = y_test_prob_SL_mean / len(y_test_prob_SL)

y_test_predicted =  np.argmax(y_test_prob_SL_mean  , axis=1   )       
y_test_predicted = y_test_predicted.astype(np.int)
cm_after_adaptation = confusion_matrix(y_test, y_test_predicted)
OA_fused_superlearner = accuracy_score(y_test,y_test_predicted)

print('---->  superLearner fusion for dataset: ' ,dataset, ' with testing set (training size ',trainSize, ' percent )' )

print('acc 1 to 5, fused acc : ') 
print(OA1); print(OA2); print(OA3); print(OA4); print(OA5);  
print(OA_fused_naive); print(OA_fused_weighted); print(OA_fused_majority); print(OA_fused_superlearner); print(maximum_accuracy)
print(np.mean(OA))
print(np.std(OA))

# for i in range(len(y_test)):
  # if y_test_predicted[i] !=y_test[i]:
      # print('image '+str(i)+ 'true label: ' + CLASS_NAMES[y_test[i]] +str(probabilities[:,y_test[i]])+ ' but majority said: ' + CLASS_NAMES[y_test_predicted[i]] + str( probabilities[:,y_test_predicted[i]]   ) )
      # print('image '+str(i)+  ' true label: ' + str(y_test[i]) +str(Test_predicted_all[i,:]   )+ ' but majority said: ' + str( y_test_predicted[i] ) + str( Test_predicted_all[i,:]    ) )

# y_prob = np.zeros(  y_test_prob1.shape  )
# for i in range(len(y_test)):
#     y_prob[i,:] = (  y_test_prob5[i,:] + y_test_prob_SL[i,:]  ) / (2)

# Test_predicted = np.argmax(y_prob, axis=1)
# cm_after_adaptation = confusion_matrix(y_test, Test_predicted)
# OA_fused = accuracy_score(y_test,Test_predicted)
# print('---->  Average prob fusion with testing size %i percent' % trainSize)
# print(OA_fused)
# print('acc 1 to 5, fused acc : ' , OA1 ,  OA2 , OA3 ,  OA4 ,OA5 ,OA_fused) 


Streaming output truncated to the last 5000 lines.
Epoch 110/300
4/4 [==============================] - 0s 4ms/step - loss: 0.3995 - accuracy: 0.8888
Epoch 111/300
4/4 [==============================] - 0s 4ms/step - loss: 0.4258 - accuracy: 0.8929
Epoch 112/300
4/4 [==============================] - 0s 5ms/step - loss: 0.4447 - accuracy: 0.8816
Epoch 113/300
4/4 [==============================] - 0s 5ms/step - loss: 0.3720 - accuracy: 0.8972
Epoch 114/300
4/4 [==============================] - 0s 4ms/step - loss: 0.4178 - accuracy: 0.8895
Epoch 115/300
4/4 [==============================] - 0s 4ms/step - loss: 0.3868 - accuracy: 0.8966
Epoch 116/300
4/4 [==============================] - 0s 4ms/step - loss: 0.4080 - accuracy: 0.8969
Epoch 117/300
4/4 [==============================] - 0s 4ms/step - loss: 0.4097 - accuracy: 0.8841
Epoch 118/300
4/4 [==============================] - 0s 4ms/step - loss: 0.3986 - accuracy: 0.9005
Epoch 119/300
4/4 [==============================] - 0s 4m

In [ ]:
y_prob_mean = np.zeros(y_prob_ensemble[0].shape)
for i in range(len(y_prob_ensemble)):
  y_prob_mean = y_prob_mean  + y_prob_ensemble[i]
  y_test_predicted =  np.argmax(y_prob_ensemble[i]  , axis=1   )  
  OA = accuracy_score(y_test,y_test_predicted); print(OA)
y_prob_mean = y_prob_mean / len(y_prob_ensemble)

y_test_predicted =  np.argmax(y_prob_mean  , axis=1   )     
# y_test_predicted = temp_result
y_test_predicted = y_test_predicted.astype(np.int)
# print(y_test,y_test_predicted)
cm_after_adaptation = confusion_matrix(y_test, y_test_predicted)
OA_fused = accuracy_score(y_test,y_test_predicted)
print('---->  superLearner fusion with testing set (training size %i percent)' % trainSize)
print(OA_fused)
print('acc 1 to 5, fused acc : ' , OA1 ,  OA2 , OA3 ,  OA4 ,OA5 ,OA_fused) 

In [ ]:
len(y_prob_ensemble)

In [ ]:
i=199
plt.imshow(X_test[i])
# plt.title(    'true label: ' + CLASS_NAMES[y_test[i]] +str(probabilities[:,y_test[i]])+ ' but majority said: ' + CLASS_NAMES[y_test_predicted[i]] + str( probabilities[:,y_test_predicted[i]])        )
plt.title(    'image '+str(i)+  ' true label: ' + str(y_test[i])+ CLASS_NAMES[y_test[i]] + ' but majority said: ' + str( y_test_predicted[i] )+ CLASS_NAMES[y_test_predicted[i]] +  str( Test_predicted_all[i,:]    ) + str(  confidence_in_predicted_class[i,:]       )         )
plt.axis('off')


In [ ]:
################################################
#  fusion based on super learner from probablities to predicted classes
from sklearn.linear_model import LogisticRegression

probabilities = np.vstack(( acc1  , acc2))
probabilities = np.vstack(( probabilities  , acc3))
probabilities = np.vstack(( probabilities  , acc4))
probabilities = np.vstack(( probabilities  , acc5))
print('acc1.shape: ' ,  acc1.shape)
probabilities1 = np.tile(acc1, [y_validation_prob1.shape[0] , 1])
probabilities2 = np.tile(acc2, [y_validation_prob2.shape[0] , 1])
probabilities3 = np.tile(acc3, [y_validation_prob3.shape[0] , 1])
probabilities4 = np.tile(acc4, [y_validation_prob4.shape[0] , 1])
probabilities5 = np.tile(acc5, [y_validation_prob5.shape[0] , 1])

print('probabilities1.shape: ' ,  probabilities1.shape)
print('probabilities1: ' ,  probabilities1)
print('probabilities2: ' ,  probabilities2)
print('probabilities3: ' ,  probabilities3)

fold_yhats =[]
fold_yhats.append(y_validation_prob1); fold_yhats.append(probabilities1)
fold_yhats.append(y_validation_prob2); fold_yhats.append(probabilities2)
fold_yhats.append(y_validation_prob3); fold_yhats.append(probabilities3)
fold_yhats.append(y_validation_prob4); fold_yhats.append(probabilities4)
fold_yhats.append(y_validation_prob5); fold_yhats.append(probabilities5)
pred_prob_outputs = np.hstack(fold_yhats)
print('pred_prob_outputs.shape: ' ,  pred_prob_outputs.shape)
# Y_val = tf.keras.utils.to_categorical(y_val , num_classes )
# Y_val = np_utils.to_categorical(y_val , num_classes )
# fit a meta model
def fit_meta_model(X, y):
	model = LogisticRegression(random_state=2020, class_weight ='balanced',  verbose=1, multi_class='auto', tol= 1e-6)
	model.fit(X, y)
	return model

def fit_my_meta_model(pred_prob_outputs, Y_val):
	num_classes = Y_val.shape[1]
	inputs = tf.keras.layers.Input(shape=( pred_prob_outputs.shape[1]))
	x = tf.keras.layers.Dense( pred_prob_outputs.shape[1] ,  activation='relu')(inputs)
	x = tf.keras.layers.BatchNormalization()(x)
	x = tf.keras.layers.Dropout(0.8)(x)
	x = tf.keras.layers.Dense( pred_prob_outputs.shape[1] ,  activation='relu')(x)
	x = tf.keras.layers.BatchNormalization()(x)
	# x = tf.keras.layers.Dropout(0.2)(x)
	x = tf.keras.layers.Dense( num_classes ,  activation='relu')(x)
	# x = tf.keras.layers.BatchNormalization()(x)
	# x = tf.keras.layers.Dropout(0.2)(x)
	# x0 = tf.keras.layers.Conv1D( filters=128 , kernel_size=1 , strides=1, activation='relu')(inputs)
	# x = tf.keras.layers.BatchNormalization()(x0)
	# x = tf.keras.layers.GlobalAveragePooling1D()(x)
	outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
	metalernet_model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
	metalernet_model.summary()

	metalernet_model.compile(loss=['categorical_crossentropy'], \
			optimizer = tf.keras.optimizers.Adam(lr=0.001),metrics=['accuracy'])
	metalernet_model.fit(pred_prob_outputs, Y_val, epochs=10)
	metalernet_model.compile(loss=['categorical_crossentropy'], \
			optimizer = tf.keras.optimizers.Adam(lr=0.0001),metrics=['accuracy'])
	metalernet_model.fit(pred_prob_outputs, Y_val, epochs=5)
	metalernet_model.compile(loss=['categorical_crossentropy'], \
			optimizer = tf.keras.optimizers.Adam(lr=0.00001),metrics=['accuracy'])
	metalernet_model.fit(pred_prob_outputs, Y_val, epochs=5)
	return metalernet_model

y_fold0_and_1_categorical = np_utils.to_categorical(y_fold0_and_1 , num_of_classes )
metalernet_model = fit_my_meta_model( pred_prob_outputs   ,  y_fold0_and_1_categorical )
temp_result = metalernet_model.predict(pred_prob_outputs)
y_validation_predicted = np.round(  np.argmax(temp_result  , axis=1   )  )
# y_validation_predicted = temp_result
y_validation_predicted = y_validation_predicted.astype(np.int)
cm_after_adaptation = confusion_matrix(y_fold0_and_1, y_validation_predicted)
OA_fused = accuracy_score(y_fold0_and_1,y_validation_predicted)
print('---->  superLearner fusion with validation set: ', OA_fused)
print('acc 1 to 5, fused acc : ' , OA1 ,  OA2 , OA3 ,  OA4 ,OA5 ,OA_fused) 

probabilities1 = np.tile(acc1, [y_test_prob1.shape[0] , 1])
probabilities2 = np.tile(acc2, [y_test_prob2.shape[0] , 1])
probabilities3 = np.tile(acc3, [y_test_prob3.shape[0] , 1])
probabilities4 = np.tile(acc4, [y_test_prob4.shape[0] , 1])
probabilities5 = np.tile(acc5, [y_test_prob5.shape[0] , 1])
fold_yhats =[]
fold_yhats.append(y_test_prob1); fold_yhats.append(probabilities1)
fold_yhats.append(y_test_prob2); fold_yhats.append(probabilities2)
fold_yhats.append(y_test_prob3); fold_yhats.append(probabilities3)
fold_yhats.append(y_test_prob4); fold_yhats.append(probabilities4)
fold_yhats.append(y_test_prob5); fold_yhats.append(probabilities5)
test_prob_outputs = np.hstack(fold_yhats)

temp_result = metalernet_model.predict(test_prob_outputs)
y_test_predicted = np.round(  np.argmax(temp_result  , axis=1   )       )
# y_test_predicted = temp_result
y_test_predicted = y_test_predicted.astype(np.int)
# print(y_test,y_test_predicted)
cm_after_adaptation = confusion_matrix(y_test, y_test_predicted)
OA_fused = accuracy_score(y_test,y_test_predicted)
print('---->  superLearner fusion with testing set (training size %i percent)' % trainSize)
print(OA_fused)
print('acc 1 to 5, fused acc : ' , OA1 ,  OA2 , OA3 ,  OA4 ,OA5 ,OA_fused) 

for i in range(len(y_test)):
  if y_test_predicted[i] !=y_test[i]:
      print('image '+str(i)+ 'true label: ' + CLASS_NAMES[y_test[i]] +str(probabilities[:,y_test[i]])+ ' but majority said: ' + CLASS_NAMES[y_test_predicted[i]] + str( probabilities[:,y_test_predicted[i]]   ) )
      # print('image '+str(i)+  ' true label: ' + str(y_test[i]) +str(Test_predicted_all[i,:]   )+ ' but majority said: ' + str( y_test_predicted[i] ) + str( Test_predicted_all[i,:]    ) )




In [ ]:
################################################
#  fusion based on super learner
probabilities1 = np.tile(acc1, [y_validation_prob1.shape[0] , 1])
probabilities2 = np.tile(acc2, [y_validation_prob2.shape[0] , 1])
probabilities3 = np.tile(acc3, [y_validation_prob3.shape[0] , 1])
probabilities4 = np.tile(acc4, [y_validation_prob4.shape[0] , 1])
probabilities5 = np.tile(acc5, [y_validation_prob5.shape[0] , 1])
validation_accuracies = np.zeros((  probabilities1.shape[0]    ,  Num_of_classifiers,   probabilities1.shape[1]  )    )
for i in range(validation_accuracies.shape[0]):
    validation_accuracies[i,0,:]  = probabilities1[i,:]
    validation_accuracies[i,1,:]  = probabilities2[i,:]
    validation_accuracies[i,2,:]  = probabilities3[i,:]
    validation_accuracies[i,3,:]  = probabilities4[i,:]
    validation_accuracies[i,4,:]  = probabilities5[i,:]
print('validation_accuracies.shape: ' ,  validation_accuracies.shape)

pred_prob_outputs = np.zeros((  y_validation_prob1.shape[0]    ,  Num_of_classifiers,   y_validation_prob1.shape[1]  )    )
for i in range(pred_prob_outputs.shape[0]):
    pred_prob_outputs[i,0,:]  = y_validation_prob1[i,:]
    pred_prob_outputs[i,1,:]  = y_validation_prob2[i,:]
    pred_prob_outputs[i,2,:]  = y_validation_prob3[i,:]
    pred_prob_outputs[i,3,:]  = y_validation_prob4[i,:]
    pred_prob_outputs[i,4,:]  = y_validation_prob5[i,:]
print('pred_prob_outputs.shape: ' ,  pred_prob_outputs.shape)

# Y_val = tf.keras.utils.to_categorical(y_val , num_classes )
y_fold0_and_1_categorical = np_utils.to_categorical(y_fold0_and_1 , num_classes )
print('y_fold0_and_1_categorical.shape: ' ,  y_fold0_and_1_categorical.shape)

inputs1 = tf.keras.layers.Input(shape=( validation_accuracies.shape[1] , validation_accuracies.shape[2]))
x0= tf.x0= tf.keras.layers.Dense( num_classes ,  activation='relu')(inputs1)
predicted_probabilities = tf.keras.layers.BatchNormalization()(x0)

# x0 = tf.keras.layers.Conv1D( filters=num_classes , kernel_size=1 , strides=1, activation='relu')(inputs1)
# predicted_probabilities = tf.keras.layers.BatchNormalization()(x0)

inputs2 = tf.keras.layers.Input(shape=( validation_accuracies.shape[1] , validation_accuracies.shape[2]))
x0= tf.keras.layers.Dense( num_classes ,  activation='relu')(inputs2)
weights= tf.keras.layers.BatchNormalization()(x0)
# x0 = keras.layers.Dense( int(num_classes/2) ,  activation='sigmoid')(x0)
# weights = tf.keras.layers.BatchNormalization()(x0)
# x0                 = tf.keras.layers.Conv1D( filters=num_classes , kernel_size=1 , strides=1, activation='relu')(inputs2)
# weights = tf.keras.layers.BatchNormalization()(x0)

x = tf.keras.layers.Multiply()([predicted_probabilities , weights])
# x = tf.keras.layers.Conv1D( filters=num_classes , kernel_size=3 , strides=1, activation='relu')(x)
x= tf.keras.layers.Dense( num_classes ,  activation='relu')(x)
# x = tf.keras.layers.BatchNormalization()(x)
# x = tf.keras.layers.GlobalAveragePooling1D()(x)
# x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Flatten()(x)

outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
metalernet_model = tf.keras.models.Model(inputs=[inputs1 , inputs2], outputs=outputs)
metalernet_model.summary()

metalernet_model.compile(loss=['categorical_crossentropy'], \
    optimizer = tf.keras.optimizers.Adam(lr=0.001),metrics=['accuracy'])
metalernet_model.fit([validation_accuracies,pred_prob_outputs], y_fold0_and_1_categorical, epochs=20)
metalernet_model.compile(loss=['categorical_crossentropy'], \
    optimizer = tf.keras.optimizers.Adam(lr=0.0001),metrics=['accuracy'])
metalernet_model.fit([validation_accuracies,pred_prob_outputs], y_fold0_and_1_categorical, epochs=10)


fused_pred_prob = metalernet_model.predict([validation_accuracies,pred_prob_outputs])
Test_predicted = np.argmax(fused_pred_prob, axis=1)
cm_after_adaptation = confusion_matrix(y_fold0_and_1, Test_predicted)
OA_fused = accuracy_score(y_fold0_and_1,Test_predicted)
print('---->  superLearner fusion on validation data with %i percent' % trainSize)
print(OA_fused)
print('acc 3 to 5, fused acc : ' , OA1 ,  OA2 , OA3 ,  OA4 ,OA5 ,OA_fused) 

probabilities1 = np.tile(acc1, [y_test_prob1.shape[0] , 1])
probabilities2 = np.tile(acc2, [y_test_prob2.shape[0] , 1])
probabilities3 = np.tile(acc3, [y_test_prob3.shape[0] , 1])
probabilities4 = np.tile(acc4, [y_test_prob4.shape[0] , 1])
probabilities5 = np.tile(acc5, [y_test_prob5.shape[0] , 1])
validation_accuracies = np.zeros((  probabilities1.shape[0]    ,  Num_of_classifiers,   probabilities1.shape[1]  )    )
for i in range(validation_accuracies.shape[0]):
    validation_accuracies[i,0,:]  = probabilities1[i,:]
    validation_accuracies[i,1,:]  = probabilities2[i,:]
    validation_accuracies[i,2,:]  = probabilities3[i,:]
    validation_accuracies[i,3,:]  = probabilities4[i,:]
    validation_accuracies[i,4,:]  = probabilities5[i,:]
print('validation_accuracies.shape: ' ,  validation_accuracies.shape)

pred_prob_outputs = np.zeros((  y_test_prob1.shape[0]    ,  Num_of_classifiers,   y_test_prob1.shape[1]  )    )
for i in range(pred_prob_outputs.shape[0]):
    pred_prob_outputs[i,0,:]  = y_test_prob1[i,:]
    pred_prob_outputs[i,1,:]  = y_test_prob2[i,:]
    pred_prob_outputs[i,2,:]  = y_test_prob3[i,:]
    pred_prob_outputs[i,3,:]  = y_test_prob4[i,:]
    pred_prob_outputs[i,4,:]  = y_test_prob5[i,:]
print('pred_prob_outputs.shape: ' ,  pred_prob_outputs.shape)

fused_pred_prob = metalernet_model.predict([validation_accuracies,pred_prob_outputs])
Test_predicted = np.argmax(fused_pred_prob, axis=1)
cm_after_adaptation = confusion_matrix(y_test, Test_predicted)
OA_fused = accuracy_score(y_test,Test_predicted)
print('---->  superLearner fusion with testing data (train size %i percent) ' % trainSize)
print(OA_fused)
print('acc 3 to 5, fused acc : ' , OA1 ,  OA2 , OA3 ,  OA4 ,OA5 ,OA_fused) 

In [ ]:
################################################
#  fusion based on super learner
from sklearn.linear_model import LogisticRegression
fold_yhats =[]
fold_yhats.append(y_validation_prob1)
fold_yhats.append(y_validation_prob2)
fold_yhats.append(y_validation_prob3)
fold_yhats.append(y_validation_prob4)
fold_yhats.append(y_validation_prob5)
pred_prob_outputs = np.hstack(fold_yhats)
# Y_val = tf.keras.utils.to_categorical(y_val , num_classes )
Y_val = np_utils.to_categorical(y_fold0_and_1 , num_classes )
# fit a meta model
def fit_meta_model(X, y):
	model = LogisticRegression(solver='liblinear')
	model.fit(X, y)
	return model
metalernet_models = []
for band in range(Y_val.shape[1]):
    M = fit_meta_model( pred_prob_outputs   ,  Y_val[:,band])
    metalernet_models.append(M)

fused_pred_prob = np.zeros(Y_val.shape)
for band in range( Y_val.shape[1]   ):
    M = metalernet_models[band]
    fused_pred_prob[:,band] = M.predict(pred_prob_outputs)
Test_predicted = np.argmax(fused_pred_prob, axis=1)
cm_after_adaptation = confusion_matrix(y_fold0_and_1, Test_predicted)
OA_fused = accuracy_score(y_fold0_and_1,Test_predicted)
print('---->  superLearner fusion with testing size %i percent' % trainSize)
print(OA_fused)
print('acc 3 to 5, fused acc : ' , OA1 ,  OA2 , OA3 ,  OA4 ,OA5 ,OA_fused) 
fold_yhats =[]
fold_yhats.append(y_test_prob1)
fold_yhats.append(y_test_prob2)
fold_yhats.append(y_test_prob3)
fold_yhats.append(y_test_prob4)
fold_yhats.append(y_test_prob5)
test_prob_outputs = np.hstack(fold_yhats)

fused_pred_prob = np.zeros(  (y_test.shape[0]  , Y_val.shape[1]))
for band in range(  Y_val.shape[1]  ):
    M = metalernet_models[band]
    fused_pred_prob[:,band] = M.predict(test_prob_outputs)
Test_predicted = np.argmax(fused_pred_prob, axis=1)
cm_after_adaptation = confusion_matrix(y_test, Test_predicted)
OA_fused = accuracy_score(y_test,Test_predicted)
print('---->  superLearner fusion with testing size %i percent' % trainSize)
print(OA_fused)
print('acc 3 to 5, fused acc : ' , OA1 ,  OA2 , OA3 ,  OA4 ,OA5 ,OA_fused) 



In [ ]:

# 5. Plot confusion matrix
OA_fused = accuracy_score(y_test,y_test_predicted)
print('---->  superLearner fusion with testing size %i percent' % trainSize)
print(OA_fused)
print('acc 1 to 5, fused acc : ' , OA1 ,  OA2 , OA3 ,  OA4 ,OA5 ,OA_fused) 
cnf_matrix =confusion_matrix(y_test,y_test_predicted)
np.set_printoptions(precision=1) 
print('Confusion matrix, without normalization')
plt.figure(figsize = (12,9))
plot_confusion_matrix2(cnf_matrix)
#plot_confusion_matrix1(cnf_matrix, classes,  normalize=False, cmap=plt.cm.Blues)
plt.show()


In [ ]:
import pathlib

data_dir = '/content/gdrive/My Drive/RS_datasets/UCMerced/Images'
# data_dir = '/content/gdrive/My Drive/RS_datasets/NWPU_RESISC45'
data_dir = pathlib.Path(data_dir)
print(data_dir)

CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"])
print(CLASS_NAMES)

CLASS_NAMES = ['freeway' , 'agricultural' ,'airplane' ,'baseballdiamond' ,'buildings' ,'beach',
 'chaparral' ,'denseresidential' ,'forest' ,'golfcourse' ,'harbor',
 'intersection', 'mediumresidential' ,'runway' , 'mobilehomepark', 'overpass',
 'parkinglot' ,'river' ,'sparseresidential' ,'storagetanks',
 'tenniscourt']
print(CLASS_NAMES)

data_dir = '/content/gdrive/My Drive/RS_datasets/optimal31'

CLASS_NAMES = [ 'airplane' ,'airport', 'aseball field', 'basketball court', 'beach', 'bridge', 'chaparral', 
'church', 'round farmland', 'business district', 'dense houses', 'desert', 'forest', 'freeway', 'golf field', 
 'playground', 'harbor', 'factory', 'crossroads', 'island', 'lake', 'meadow', 'medium houses', 'mobile house area', 
 'mountain', 'overpass', 'parking lot', 'railway', 'square farmland', 'roundabout', 'runway' ]

# (0) airplane; (1) airport; (2) baseball field; (3) basketball court; (4) beach; (5) bridge; (6) chaparral; 
# (7) church; (8) round farmland; (9) business district; (10) dense houses; (11) desert; (12) forest; 
# (13) freeway; (14) golf field; (15) playground; (16) harbor; (17) factory; (18) crossroads; (19) island; (20) lake; 
# (21) meadow; (22) medium houses; (23) mobile house area; (24) mountain; (25) overpass; (26) parking lot; 
# (27) railway; (28) square farmland; (29) roundabout; (30) runway.

def show_batch(image_batch, label_batch_true, label_batch_pred, CLASS_NAMES):
  plt.figure(figsize=(20,20))
  for n in range(len(label_batch_true)):
      ax = plt.subplot(5,5,n+1)
      plt.imshow(image_batch[n])
      plt.title(  'true: ' +  CLASS_NAMES[label_batch_true[n]] + 'pred: ' + CLASS_NAMES[  label_batch_pred[n] ]   )
      plt.axis('off')

missclassified_images = []
for i in range( len(y_test) ):
  if y_test[i] != y_test_predicted[i]:
    missclassified_images.append(i)

missclassified_images = np.asarray(missclassified_images)
print(missclassified_images)

image_batch = X_test[missclassified_images]
label_batch_true = y_test[missclassified_images]
label_batch_pred = y_test_predicted[missclassified_images]
print('image_batch.shape: ' , image_batch.shape)
print('label_batch_true.shape: ' , label_batch_true.shape)
print('label_batch_pred.shape: ' , label_batch_pred.shape)
show_batch(image_batch, label_batch_true, label_batch_pred, CLASS_NAMES)



